In [154]:
import pandas as pd
import numpy as np
from collections import Counter
import datetime
import re

DataFrame_Raw = pd.read_excel(r'/Users/Riccardo/TriesteProject/Covid19/Datasets/SecondWave_Citofluorimetria/citofluorimetria_COVID_2.xlsx', engine='openpyxl')

#DataFrame_Raw.iloc[0:20, :].style
#DataFrame_Raw['Decesso'].value_counts()
#DataFrame_Raw.loc[0:20, :].style

## Dataset: quick look ##

In [173]:
print('Dataset shape:\n\n', DataFrame_Raw.shape, '\n\n')
df_numeric_info = pd.DataFrame(DataFrame_Raw.count(axis=0, level=None, numeric_only=True), columns=['Numerical'])
print('N. numerical values per column:\n\n', df_numeric_info, '\n\n')
df_nans_info = pd.DataFrame(DataFrame_Raw.isna().sum().values, columns=['Nans'])
print('N. nans per column:\n\n', df_nans_info, '\n\n')
deaths_counter = Counter(DataFrame_Raw['morto_1_vivo_0'])
print('Deaths:\n\n', deaths_counter, '- 1:', 100*(deaths_counter[1]/(deaths_counter[0]+deaths_counter[1])), '%', '\n\n')
#DataFrame_Raw.loc[0:20, :].style

Dataset shape:

 (783, 38) 


N. numerical values per column:

                    Numerical
morto_1_vivo_0           783
OS_days                  783
eta_paziente             783
sesso                    783
WBC/uL                   782
Mono/uL                  770
Linfo/uL                 783
T CD3 %                  783
T CD3/uL                 783
T CD4 %                  783
T CD4/uL                 783
T CD8 %                  783
T CD8/uL                 783
CD4/CD8                  783
NK %                     783
NK/uL                    783
B CD19 %                 783
B CD19/uL                783
T CD3/HLADR %            782
T CD3 HLA DR/uL          782
T CD4 HLADR %            782
% T CD4 HLADR POS        782
T CD8 HLADR %            782
% T CD8 HLADR POS        782
T NK-like %              758
T NL-like/uL               0
LRTE % dei CD4           775
LRTE/uL                  776
Mono DR %                769
MONO DR IFI              769
Unnamed: 34                0
Unnamed:

## Age distribution ##

In [196]:
Ages = DataFrame_Raw.loc[:, 'eta_paziente'].copy()
Deaths = DataFrame_Raw.loc[:, 'morto_1_vivo_0'].copy()
mask_age = Ages.values>=70
np.sum(mask_age), np.sum(Deaths.values[mask_age==False])

(386, 24)

## Dates to reals ##

In [164]:
## columns with dates
DataFrame_V1 = DataFrame_Raw.copy()
mask_dates = np.zeros(DataFrame_V1.shape[1])
dates = []
for i, element in enumerate(DataFrame_V1.columns):
    mask_dates[i] = False
    if ('data' in element):
        mask_dates[i] = True
        dates.append(element)
        #print(element)

print('Dates in dataset:\n\n', dates, '\n')
        
## check future dates
for i in range(DataFrame_V1.shape[0]):
    for date in dates:
        element = DataFrame_V1.loc[i, date]
        if (element.year > 2021):
            print(i, date, element.year)
            
## pre-process dates
#ReferenceTime = datetime.datetime(1970, 1, 1, 1, 0)
ReferenceTime = datetime.datetime(2100, 1, 1, 1, 0)
ReferenceTime = ReferenceTime.timestamp()

for i in range(DataFrame_V1.shape[0]):
    for date in dates:
        date_time_str = DataFrame_V1.loc[i, date]
        #print(date_time_str)
        if date_time_str:
            date_time_obj = datetime.datetime.strptime(str(date_time_str), '%Y-%m-%d %H:%M:%S')
            DataFrame_V1.loc[i, date] = float(ReferenceTime - date_time_obj.timestamp())

for date in dates:
    DataFrame_V1.loc[:, date] = pd.to_numeric(DataFrame_V1.loc[:, date])
#DataFrame_V1.loc[0:20, :].style

Dates in dataset:

 ['data_accettazione', 'data_decesso', 'data_nascita'] 



## Remove columns ##

In [175]:
## drop provenienza
DataFrame_V2 = DataFrame_V1.copy()
DataFrame_V2.drop(columns=['provenienza'], inplace=True)
DataFrame_V2.shape

## drop columns with too many nans
n_nans_threshold = DataFrame_V2.shape[0] / 2.
mask_columns = DataFrame_V2.count(axis=0, level=None, numeric_only=True).values < n_nans_threshold
mask_columns
DataFrame_V2.drop(columns=DataFrame_V2.columns[mask_columns], inplace=True)

## check
df_numeric_info = pd.DataFrame(DataFrame_V2.count(axis=0, level=None, numeric_only=True), columns=['Numerical'])
print('N. numerical values per column:\n\n', df_numeric_info, '\n\n')

N. numerical values per column:

                    Numerical
data_accettazione        783
data_decesso             783
morto_1_vivo_0           783
OS_days                  783
data_nascita             783
eta_paziente             783
sesso                    783
WBC/uL                   782
Mono/uL                  770
Linfo/uL                 783
T CD3 %                  783
T CD3/uL                 783
T CD4 %                  783
T CD4/uL                 783
T CD8 %                  783
T CD8/uL                 783
CD4/CD8                  783
NK %                     783
NK/uL                    783
B CD19 %                 783
B CD19/uL                783
T CD3/HLADR %            782
T CD3 HLA DR/uL          782
T CD4 HLADR %            782
% T CD4 HLADR POS        782
T CD8 HLADR %            782
% T CD8 HLADR POS        782
T NK-like %              758
LRTE % dei CD4           775
LRTE/uL                  776
Mono DR %                769
MONO DR IFI              769 




## Insert ID column ##

In [181]:
id_values = list(map(lambda x: str(x), np.arange(1, DataFrame_Raw.shape[0]+1, 1)))

In [183]:
DataFrame_V3 = DataFrame_V2.copy()
DataFrame_V3.insert(loc=DataFrame_V3.shape[1], column='ID', value=id_values)
DataFrame_V3.count(axis=0, level=None, numeric_only=False)

data_accettazione    783
data_decesso         783
morto_1_vivo_0       783
OS_days              783
data_nascita         783
eta_paziente         783
sesso                783
WBC/uL               782
Mono/uL              770
Linfo/uL             783
T CD3 %              783
T CD3/uL             783
T CD4 %              783
T CD4/uL             783
T CD8 %              783
T CD8/uL             783
CD4/CD8              783
NK %                 783
NK/uL                783
B CD19 %             783
B CD19/uL            783
T CD3/HLADR %        782
T CD3 HLA DR/uL      782
T CD4 HLADR %        782
% T CD4 HLADR POS    782
T CD8 HLADR %        782
% T CD8 HLADR POS    782
T NK-like %          758
LRTE % dei CD4       775
LRTE/uL              776
Mono DR %            769
MONO DR IFI          769
ID                   783
dtype: int64

## Save dataset ##

In [182]:
DataFrame_V3.to_csv(r'/Users/Riccardo/TriesteProject/Covid19/Datasets/SecondWave_Citofluorimetria/citofluorimetria_COVID_2_cleaned.csv', index = False)